# Big Data and Large-Scale Computing

## Introduction to PySpark and RDDs

The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data. We will write code that calculates the most common words in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page).

This could also be scaled to find the most common words in Wikipedia.


## Part 1: Creating a base RDD and pair RDDs

In this part, we will explore creating a base RDD with `parallelize` and using pair RDDs to count words.

Let's first start by creating the SparkContext

In [1]:
import sys
sys.path.append("/opt/packages/spark/latest/python/lib/py4j-0.10.9-src.zip")
sys.path.append("/opt/packages/spark/latest/python/")
sys.path.append("/opt/packages/spark/latest/python/pyspark")
from pyspark import SparkConf, SparkContext
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

### (1a) Create a base RDD
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

In [2]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
print (type(wordsRDD))

<class 'pyspark.rdd.RDD'>


### (1b) Pluralize

Let's use a `map()` transformation to add the letter 's' to each string in the base RDD we just created. We'll define a Python function that returns the word with an 's' at the end of the word.  



In [3]:

def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  No attempt is made to follow proper
        pluralization rules.

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word+"s"

print (makePlural('cat'))

cats


### (1c) Apply `makePlural` to the base RDD

Now pass each item in the base RDD into a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `makePlural()` function to each element. And then call the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action to see the transformed RDD.

In [4]:

pluralRDD = wordsRDD.map(makePlural)
print (pluralRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


### (1d) Pass a `lambda` function to `map`

`lambda` functions in Python are anonymous functions that are created for one-time use. Let's create the same RDD using a `lambda` function.

In [5]:

pluralLambdaRDD = wordsRDD.map(lambda x:x+'s')
print (pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


### (1e) Length of each word

Now use `map()` and a `lambda` function to return the number of characters in each word.  We'll `collect` this result directly into a variable.

In [6]:

pluralLengths = (pluralRDD
                 .map(lambda x:len(x))
                 .collect())
print (pluralLengths)

[4, 9, 4, 4, 4]


### (1f) Pair RDDs

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple `(k, v)` where `k` is the key and `v` is the value. In this example, we will create a pair consisting of `('<word>', 1)` for each word element in the RDD.
We can create the pair RDD using the `map()` transformation with a `lambda()` function to create a new RDD.

In [7]:

wordPairs = wordsRDD.map(lambda x:(x,1))
print (wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


## Part 2: Counting with pair RDDs

Now, let's count the number of times a particular word appears in the RDD. There are multiple ways to perform the counting, but some are much less efficient than others.

A naive approach would be to `collect()` all of the elements and count them in the driver program. While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use data parallel operations.

### (2a) `groupByKey()` approach
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions.

There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.

Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`.

In [8]:

# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print ('{0}: {1}'.format(key, list(value)))

cat: [1, 1]
elephant: [1]
rat: [1, 1]


### (2b) Use `groupByKey()` to obtain the counts

Using the `groupByKey()` transformation creates an RDD containing 3 elements, each of which is a pair of a word and a Python iterator.

Now sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [9]:

wordCountsGrouped = wordsGrouped.map(lambda x: (x[0],sum(x[1])))
print (wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (2c) Counting using `reduceByKey`

A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

In [10]:

# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCountsGrouped = wordPairs.reduceByKey(lambda x,y:x+y)
print (wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (2d) All together

The expert version of the code performs the `map()` to pair RDD, `reduceByKey()` transformation, and `collect` in one statement.

In [11]:

wordCountsCollected = (wordsRDD
                       .map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)
                       .collect())
print (wordCountsCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


## Part 3: Finding unique words and a mean value

### (3a) Unique words

Calculate the number of unique words in `wordsRDD`.  You can use other RDDs that you have already created to make this easier.

In [12]:

uniqueWords = [k for k,c in wordCountsCollected]
print (uniqueWords)
print (len(uniqueWords))

['cat', 'elephant', 'rat']
3


### (3b) Mean using `reduce`

Find the mean number of words per unique word in `wordPairs`.

Use a `reduce()` action and the imported `add` function to sum the counts in `wordPairs` and then divide by the number of unique words.  First `map()` the pair RDD `wordPairs`, which consists of (key, value) pairs, to an RDD of values.

In [13]:

from operator import add
totalCount = (wordPairs
              .map(lambda x:x[1])
              .reduce(add))
average = totalCount / float(len(uniqueWords))
print (totalCount)
print (round(average, 2))

5
1.67


## Part 4: Apply word count to a file

In this section we will finish developing our word count application.  We'll have to build the `wordCount` function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

### (4a) `wordCount` function

First, define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this assignment.  This function should take in an RDD that is a list of words like `wordsRDD` and return a pair RDD that has all of the words and their associated counts.

In [14]:
 
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    wordGroup=wordListRDD.map(lambda x:(x,1))
    wordCnt=wordGroup.reduceByKey(lambda a,b:a+b)
    return wordCnt
print (wordCount(wordsRDD).collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (4b) Capitalization and punctuation

Real world files are more complicated than the data we have been using thus far. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.

Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.
If you are unfamiliar with regular expressions, you may want to review [this tutorial](https://developers.google.com/edu/python/regular-expressions) from Google.  Also, [this website](https://regex101.com/#python) is  a great resource for debugging your regular expression.

In [15]:

import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    outputString=re.sub(r'[^0-9a-zA-Z\s]','',text.lower())
    outputString=outputString.strip()
    return outputString
print (removePunctuation('Hi, you!'))
print (removePunctuation(' No under_score!'))
print (removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


### (4c) Load a text file

For the next part of this homework, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lower case.  Since the file is large we use `take(15)`, so that we only print 15 lines.

In [16]:
# Just run this code

fileName = 'file:///ocean/projects/cie170025p/shared/data/shakespeare.txt'

shakespeareRDD = sc.textFile(fileName, 8).map(removePunctuation)
sampleLines = ('\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda t: '{0}: {1}'.format(t[1], t[0]))  # to 'lineNum: line'
                .take(15)))
print (sampleLines)

0: the project gutenberg ebook of the complete works of william shakespeare by
1: william shakespeare
2: 
3: this ebook is for the use of anyone anywhere at no cost and with
4: almost no restrictions whatsoever  you may copy it give it away or
5: reuse it under the terms of the project gutenberg license included
6: with this ebook or online at wwwgutenbergorg
7: 
8: this is a copyrighted project gutenberg ebook details below
9: please follow the copyright guidelines in this file
10: 
11: title the complete works of william shakespeare
12: 
13: author william shakespeare
14: 


### (4d) Words from lines

Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that we need to split each line by its spaces. ** Performed in (4d). **
  + The second issue is we need to filter out empty lines. ** Performed in (4e). **

Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

> Note:
> * Do not use the default implementation of `split()`, but pass in a separator value.  For example, to split `line` by commas you would use `line.split(',')`.

In [17]:

shakespeareWordsRDD = shakespeareRDD.flatMap(lambda line:line.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()
print (shakespeareWordsRDD.top(5))
print (shakespeareWordCount)

['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds']
949103


### (4e) Remove empty elements

The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [18]:

shakeWordsRDD = shakespeareWordsRDD.filter(lambda word:word!="")
shakeWordCount = shakeWordsRDD.count()
print (shakeWordCount)
#print(type(shakeWordsRDD))

903731


### (4f) Count the words

We now have an RDD that is only words.  Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.

You'll notice that many of the words are common English words. These are called stopwords. Use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [19]:

top15WordsAndCounts = wordCount(shakeWordsRDD).takeOrdered(15,lambda x:-x[1])
print ('\n'.join(map(lambda t: '{0}: {1}'.format(t[0], t[1]), top15WordsAndCounts)))

the: 27825
and: 26791
i: 20681
to: 19262
of: 18289
a: 14667
you: 13716
my: 12480
that: 11135
in: 11027
is: 9621
not: 8745
for: 8261
with: 8046
me: 7768


### The End